# Trexquant Interview Project (The Hangman Game)

* Copyright Trexquant Investment LP. All Rights Reserved. 
* Redistribution of this question without written consent from Trexquant is prohibited

## Instruction:
For this coding test, your mission is to write an algorithm that plays the game of Hangman through our API server. 

When a user plays Hangman, the server first selects a secret word at random from a list. The server then returns a row of underscores (space separated)—one for each letter in the secret word—and asks the user to guess a letter. If the user guesses a letter that is in the word, the word is redisplayed with all instances of that letter shown in the correct positions, along with any letters correctly guessed on previous turns. If the letter does not appear in the word, the user is charged with an incorrect guess. The user keeps guessing letters until either (1) the user has correctly guessed all the letters in the word
or (2) the user has made six incorrect guesses.

You are required to write a "guess" function that takes current word (with underscores) as input and returns a guess letter. You will use the API codes below to play 1,000 Hangman games. You have the opportunity to practice before you want to start recording your game results.

Your algorithm is permitted to use a training set of approximately 250,000 dictionary words. Your algorithm will be tested on an entirely disjoint set of 250,000 dictionary words. Please note that this means the words that you will ultimately be tested on do NOT appear in the dictionary that you are given. You are not permitted to use any dictionary other than the training dictionary we provided. This requirement will be strictly enforced by code review.

You are provided with a basic, working algorithm. This algorithm will match the provided masked string (e.g. a _ _ l e) to all possible words in the dictionary, tabulate the frequency of letters appearing in these possible words, and then guess the letter with the highest frequency of appearence that has not already been guessed. If there are no remaining words that match then it will default back to the character frequency distribution of the entire dictionary.

This benchmark strategy is successful approximately 18% of the time. Your task is to design an algorithm that significantly outperforms this benchmark.

In [1]:
import torch
import numpy as np
import random
import torch.nn as nn
from tqdm import tqdm
from math import floor, log2
import json
print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0))

True
NVIDIA GeForce RTX 4090


In [26]:
with open("words_250000_train.txt", 'r') as f:
    words = [line.strip() for line in f if line.strip()]
words = [w for w in words if len(w) <= 20]

In [27]:
def simulate_state(word: str, bitmask: int):
    unique_letters = sorted(set(word))
    guessed_letters = {
        unique_letters[i]
        for i in range(len(unique_letters))
        if (bitmask >> i) & 1
    }
    masked = [c if c in guessed_letters else '_' for c in word]
    label = set([c for c in word if c not in guessed_letters and c != '_'])
    return {
        'masked_word': masked,
        'label': label,
    }

In [28]:
with open("tokenized_words_1_masked_letter.jsonl", "w") as f:
    for word in words:
        k = len(set(word))
        for i in range(k):
            bitmask = 2**k-2**i-1
            result = simulate_state(word, bitmask)
            result["label"] = list(result["label"])
            f.write(json.dumps(result) + "\n")

In [29]:
def encode_state(state, max_length=20):
    char_to_id = {c: idx for idx, c in enumerate("abcdefghijklmnopqrstuvwxyz")}
    unguessed = 26
    pad = 27
    
    input_id = [
        char_to_id[c] if c in char_to_id else unguessed
        for c in state['masked_word']
    ]
    if len(input_id) < max_length:
        input_id += [pad] * (max_length - len(input_id))
    else:
        input_id = input_id[:max_length]
    
    label_id = [0] * 26
    for c in state['label']:
        label_id[char_to_id[c]] = 1

    return input_id, label_id

In [30]:
with open("tokenized_words_1_masked_letter.jsonl", "r") as f:
    N = sum(1 for _ in f)

print(N)  

1679114


In [31]:
encoded_words = np.zeros((N, 20), dtype=np.uint8)
encoded_labels = np.zeros((N, 26), dtype=np.uint8)
attention_masks = np.zeros((N, 20), dtype=np.uint8)

with open("tokenized_words_1_masked_letter.jsonl", "r") as f:
    for i, line in enumerate(f):
        state = json.loads(line)
        input_id, label_id = encode_state(state)
        mask = [0 if token == 27 else 1 for token in input_id]

        encoded_words[i] = input_id
        encoded_labels[i] = label_id
        attention_masks[i] = mask

In [32]:
input_tensor = torch.tensor(encoded_words, dtype=torch.long)
label_tensor = torch.tensor(encoded_labels, dtype=torch.float64)
mask_tensor = torch.tensor(attention_masks, dtype=torch.bool)

print("Input Tensor:", input_tensor.shape)
print("Label Tensor:", label_tensor.shape)
print("Mask Tensor:", mask_tensor.shape)

Input Tensor: torch.Size([1679114, 20])
Label Tensor: torch.Size([1679114, 26])
Mask Tensor: torch.Size([1679114, 20])


In [33]:
from sklearn.model_selection import train_test_split

train_inputs, val_inputs, train_labels, val_labels, train_masks, val_masks = train_test_split(
    input_tensor, label_tensor, mask_tensor, test_size=0.1, random_state=42
)

In [34]:
from torch.utils.data import TensorDataset

train_dataset = TensorDataset(train_inputs, train_masks, train_labels)
val_dataset = TensorDataset(val_inputs, val_masks, val_labels)

In [35]:
from torch.utils.data import DataLoader

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64)

In [2]:
class HangmanTransformer(nn.Module):
    def __init__(self, vocab_size=28, embed_dim=128, num_heads=4, num_layers=6, max_seq_len=20, num_classes=26):
        super().__init__()
        
        self.token_embedding = nn.Embedding(vocab_size, embed_dim)
        self.pos_embedding = nn.Embedding(max_seq_len, embed_dim)

        encoder_layer = nn.TransformerEncoderLayer(d_model=embed_dim, nhead=num_heads, batch_first=True)
        self.transformer = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)

        self.classifier = nn.Linear(embed_dim, num_classes)

    def forward(self, input_ids, attention_mask):
        positions = torch.arange(input_ids.size(1), device=input_ids.device)
        positions = positions.unsqueeze(0).expand(input_ids.size(0), -1)
        
        x = self.token_embedding(input_ids) + self.pos_embedding(positions)

        src_key_padding_mask = ~attention_mask.bool()
        x = self.transformer(x, src_key_padding_mask=src_key_padding_mask)

        mask = attention_mask.unsqueeze(-1)
        x = (x * mask).sum(1) / mask.sum(1)

        logits = self.classifier(x)
        return logits

In [37]:
model = HangmanTransformer()
model.load_state_dict(torch.load("hangman_model_multilabel_6.pt", weights_only=True))
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)
criterion = torch.nn.BCEWithLogitsLoss()

In [38]:
def train_one_epoch(model, dataloader):
    model.train()
    total_loss, correct, total = 0, 0, 0

    for input_ids, attention_mask, labels in tqdm(dataloader, desc="Training"):
        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()
        logits = model(input_ids, attention_mask)
        loss = criterion(logits, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item() * input_ids.size(0)
        preds = logits.argmax(dim=1)
        correct += (labels.gather(1, preds.unsqueeze(1)) == 1).sum().item()
        total += labels.size(0)

    avg_loss = total_loss / total
    accuracy = correct / total
    return avg_loss, accuracy

def validate(model, dataloader):
    model.eval()
    total_loss, correct, total = 0, 0, 0

    with torch.no_grad():
        for input_ids, attention_mask, labels in tqdm(dataloader, desc="Validation"):
            input_ids = input_ids.to(device)
            attention_mask = attention_mask.to(device)
            labels = labels.to(device)

            logits = model(input_ids, attention_mask)
            loss = criterion(logits, labels)

            total_loss += loss.item() * input_ids.size(0)
            preds = logits.argmax(dim=1)
            correct += (labels.gather(1, preds.unsqueeze(1)) == 1).sum().item()
            total += labels.size(0)

    avg_loss = total_loss / total
    accuracy = correct / total
    return avg_loss, accuracy

In [39]:
for epoch in range(1, 6):
    train_loss, train_acc = train_one_epoch(model, train_loader)
    val_loss, val_acc = validate(model, val_loader)

    print(f"\nEpoch {epoch}")
    print(f"Train     | Loss: {train_loss:.4f}  Acc: {train_acc:.4f}")
    print(f"Validation| Loss: {val_loss:.4f}  Acc: {val_acc:.4f}\n")

Validation: 100%|█████████████████████████| 2624/2624 [00:12<00:00, 207.72it/s]



Epoch 1
Train     | Loss: 0.0522  Acc: 0.7176
Validation| Loss: 0.0465  Acc: 0.7462



Validation: 100%|█████████████████████████| 2624/2624 [00:11<00:00, 226.62it/s]



Epoch 2
Train     | Loss: 0.0495  Acc: 0.7279
Validation| Loss: 0.0453  Acc: 0.7514



Validation: 100%|█████████████████████████| 2624/2624 [00:12<00:00, 210.91it/s]



Epoch 3
Train     | Loss: 0.0483  Acc: 0.7334
Validation| Loss: 0.0446  Acc: 0.7539



Validation: 100%|█████████████████████████| 2624/2624 [00:12<00:00, 206.57it/s]



Epoch 4
Train     | Loss: 0.0474  Acc: 0.7384
Validation| Loss: 0.0441  Acc: 0.7572



Validation: 100%|█████████████████████████| 2624/2624 [00:12<00:00, 216.67it/s]


Epoch 5
Train     | Loss: 0.0466  Acc: 0.7416
Validation| Loss: 0.0436  Acc: 0.7599



In [40]:
torch.save(model.state_dict(), "hangman_model_multilabel_6_fine_tuned.pt")

In [3]:
model = HangmanTransformer()
model.load_state_dict(torch.load("hangman_model_multilabel_6_fine_tuned.pt", weights_only=True))
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()

HangmanTransformer(
  (token_embedding): Embedding(28, 128)
  (pos_embedding): Embedding(20, 128)
  (transformer): TransformerEncoder(
    (layers): ModuleList(
      (0-5): 6 x TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=128, out_features=128, bias=True)
        )
        (linear1): Linear(in_features=128, out_features=2048, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
        (linear2): Linear(in_features=2048, out_features=128, bias=True)
        (norm1): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
        (norm2): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
        (dropout1): Dropout(p=0.1, inplace=False)
        (dropout2): Dropout(p=0.1, inplace=False)
      )
    )
  )
  (classifier): Linear(in_features=128, out_features=26, bias=True)
)

In [4]:
import json
import requests
import random
import string
import secrets
import time
import re
import collections

try:
    from urllib.parse import parse_qs, urlencode, urlparse
except ImportError:
    from urlparse import parse_qs, urlparse
    from urllib import urlencode

from requests.packages.urllib3.exceptions import InsecureRequestWarning

requests.packages.urllib3.disable_warnings(InsecureRequestWarning)

In [16]:
class HangmanAPI(object):
    def __init__(self, access_token=None, session=None, timeout=None):
        self.hangman_url = self.determine_hangman_url()
        self.access_token = access_token
        self.session = session or requests.Session()
        self.timeout = timeout
        self.guessed_letters = []
        
        full_dictionary_location = "words_250000_train.txt"
        self.full_dictionary = self.build_dictionary(full_dictionary_location)        
        self.full_dictionary_common_letter_sorted = collections.Counter("".join(self.full_dictionary)).most_common()
        
        self.current_dictionary = []
        
    @staticmethod
    def determine_hangman_url():
        links = ['https://trexsim.com']

        data = {link: 0 for link in links}

        for link in links:

            requests.get(link)

            for i in range(10):
                s = time.time()
                requests.get(link)
                data[link] = time.time() - s

        link = sorted(data.items(), key=lambda x: x[1])[0][0]
        link += '/trexsim/hangman'
        return link

    def guess(self, word): # word input example: "_ p p _ e "
        letters = word.strip().split()
        guessed_letters = self.guessed_letters

        #this is given by chatgpt
        freq_by_length = {
            1: ['a', 'i', 'o', 'e', 'u', 'y'],
            2: ['a', 'e', 'o', 'i', 'n', 's'],
            3: ['e', 'a', 'o', 'r', 'i', 'n'],
            4: ['e', 'a', 'r', 'o', 't', 'i'],
            5: ['e', 'a', 'r', 'o', 't', 'i'],
            6: ['e', 'a', 'r', 'n', 't', 'i'],
            7: ['e', 'a', 'n', 'r', 't', 'i'],
            8: ['e', 'n', 'a', 'r', 'i', 't'],
            9: ['e', 'n', 'a', 'i', 'o', 'r'],
            10: ['e', 'n', 'a', 'r', 'i', 'o'],
            11: ['e', 'n', 't', 'a', 'r', 'i'],
            12: ['e', 'n', 't', 'a', 'i', 'r'],
            13: ['e', 'n', 't', 'a', 'r', 'i'],
            14: ['e', 'n', 't', 'a', 'i', 'r'],
            15: ['e', 'n', 't', 'a', 'r', 'i'],
            16: ['e', 'n', 't', 'a', 'i', 'r'],
            17: ['e', 'n', 't', 'a', 'r', 'i'],
            18: ['e', 'n', 't', 'a', 'i', 'r'],
            19: ['e', 'n', 't', 'a', 'r', 'i'],
            20: ['e', 'n', 't', 'a', 'i', 'r']
        }

        word_len = len(letters)

        if all(c == '_' for c in letters):
            for letter in freq_by_length[min(word_len, 20)]:
                if letter not in guessed_letters:
                    return letter
        
        char_to_id = {c: idx for idx, c in enumerate("abcdefghijklmnopqrstuvwxyz")}
        unguessed = 26
        pad = 27     
        max_length = 20

        input_id = [
                char_to_id[c] if c in char_to_id else unguessed
                for c in letters
            ]
        if len(input_id) < max_length:
            input_id += [pad] * (max_length - len(input_id))
        else:
            input_id = input_id[:max_length]
        attention_mask = [1 if t != pad else 0 for t in input_id]

        input_tensor = torch.tensor([input_id], dtype=torch.long).to(device)
        mask_tensor = torch.tensor([attention_mask], dtype=torch.bool).to(device)

        with torch.no_grad():
            logits = model(input_tensor, mask_tensor)

            for c in guessed_letters:
                if c in char_to_id:
                    logits[0, char_to_id[c]] = -float('inf')

            pred = logits.argmax(dim=1).item()
            return chr(pred + ord('a'))

    ##########################################################
    # You'll likely not need to modify any of the code below #
    ##########################################################
    
    def build_dictionary(self, dictionary_file_location):
        text_file = open(dictionary_file_location,"r")
        full_dictionary = text_file.read().splitlines()
        text_file.close()
        return full_dictionary
                
    def start_game(self, practice=True, verbose=True):
        # reset guessed letters to empty set and current plausible dictionary to the full dictionary
        self.guessed_letters = []
        self.current_dictionary = self.full_dictionary
                         
        response = self.request("/new_game", {"practice":practice})
        if response.get('status')=="approved":
            game_id = response.get('game_id')
            word = response.get('word')
            tries_remains = response.get('tries_remains')
            if verbose:
                print("Successfully start a new game! Game ID: {0}. # of tries remaining: {1}. Word: {2}.".format(game_id, tries_remains, word))
            while tries_remains>0:
                # get guessed letter from user code
                guess_letter = self.guess(word)
                    
                # append guessed letter to guessed letters field in hangman object
                self.guessed_letters.append(guess_letter)
                if verbose:
                    print("Guessing letter: {0}".format(guess_letter))
                    
                try:    
                    res = self.request("/guess_letter", {"request":"guess_letter", "game_id":game_id, "letter":guess_letter})
                except HangmanAPIError:
                    print('HangmanAPIError exception caught on request.')
                    continue
                except Exception as e:
                    print('Other exception caught on request.')
                    raise e
               
                if verbose:
                    print("Sever response: {0}".format(res))
                status = res.get('status')
                tries_remains = res.get('tries_remains')
                if status=="success":
                    if verbose:
                        print("Successfully finished game: {0}".format(game_id))
                    return True
                elif status=="failed":
                    reason = res.get('reason', '# of tries exceeded!')
                    if verbose:
                        print("Failed game: {0}. Because of: {1}".format(game_id, reason))
                    return False
                elif status=="ongoing":
                    word = res.get('word')
        else:
            if verbose:
                print("Failed to start a new game")
        return status=="success"
        
    def my_status(self):
        return self.request("/my_status", {})
    
    def request(
            self, path, args=None, post_args=None, method=None):
        if args is None:
            args = dict()
        if post_args is not None:
            method = "POST"

        # Add `access_token` to post_args or args if it has not already been
        # included.
        if self.access_token:
            # If post_args exists, we assume that args either does not exists
            # or it does not need `access_token`.
            if post_args and "access_token" not in post_args:
                post_args["access_token"] = self.access_token
            elif "access_token" not in args:
                args["access_token"] = self.access_token

        time.sleep(0.2)

        num_retry, time_sleep = 50, 2
        for it in range(num_retry):
            try:
                response = self.session.request(
                    method or "GET",
                    self.hangman_url + path,
                    timeout=self.timeout,
                    params=args,
                    data=post_args,
                    verify=False
                )
                break
            except requests.HTTPError as e:
                response = json.loads(e.read())
                raise HangmanAPIError(response)
            except requests.exceptions.SSLError as e:
                if it + 1 == num_retry:
                    raise
                time.sleep(time_sleep)

        headers = response.headers
        if 'json' in headers['content-type']:
            result = response.json()
        elif "access_token" in parse_qs(response.text):
            query_str = parse_qs(response.text)
            if "access_token" in query_str:
                result = {"access_token": query_str["access_token"][0]}
                if "expires" in query_str:
                    result["expires"] = query_str["expires"][0]
            else:
                raise HangmanAPIError(response.json())
        else:
            raise HangmanAPIError('Maintype was not text, or querystring')

        if result and isinstance(result, dict) and result.get("error"):
            raise HangmanAPIError(result)
        return result
    
class HangmanAPIError(Exception):
    def __init__(self, result):
        self.result = result
        self.code = None
        try:
            self.type = result["error_code"]
        except (KeyError, TypeError):
            self.type = ""

        try:
            self.message = result["error_description"]
        except (KeyError, TypeError):
            try:
                self.message = result["error"]["message"]
                self.code = result["error"].get("code")
                if not self.type:
                    self.type = result["error"].get("type", "")
            except (KeyError, TypeError):
                try:
                    self.message = result["error_msg"]
                except (KeyError, TypeError):
                    self.message = result

        Exception.__init__(self, self.message)

# API Usage Examples

## To start a new game:
1. Make sure you have implemented your own "guess" method.
2. Use the access_token that we sent you to create your HangmanAPI object. 
3. Start a game by calling "start_game" method.
4. If you wish to test your function without being recorded, set "practice" parameter to 1.
5. Note: You have a rate limit of 20 new games per minute. DO NOT start more than 20 new games within one minute.

In [17]:
api = HangmanAPI(access_token="a848c1cbd6ffd0bea60f09717a490b", timeout=2000)


## Playing practice games:
You can use the command below to play up to 100,000 practice games.

In [13]:
for i in range(3):
    api.start_game(practice=1,verbose=True)
    [total_practice_runs,total_recorded_runs,total_recorded_successes,total_practice_successes] = api.my_status() # Get my game stats: (# of tries, # of wins)
    practice_success_rate = total_practice_successes / total_practice_runs
    print('run %d practice games out of an allotted 100,000. practice success rate so far = %.3f' % (total_practice_runs, practice_success_rate))


Successfully start a new game! Game ID: b293fca8f0ab. # of tries remaining: 6. Word: _ _ _ _ _ _ _ _ _ _ _ .
Guessing letter: e
Sever response: {'game_id': 'b293fca8f0ab', 'status': 'ongoing', 'tries_remains': 6, 'word': '_ _ e _ _ _ e _ _ _ e '}
Guessing letter: l
Sever response: {'game_id': 'b293fca8f0ab', 'status': 'ongoing', 'tries_remains': 6, 'word': '_ _ e _ _ l e _ _ _ e '}
Guessing letter: t
Sever response: {'game_id': 'b293fca8f0ab', 'status': 'ongoing', 'tries_remains': 6, 'word': '_ _ e t t l e _ _ _ e '}
Guessing letter: s
Sever response: {'game_id': 'b293fca8f0ab', 'status': 'ongoing', 'tries_remains': 5, 'word': '_ _ e t t l e _ _ _ e '}
Guessing letter: p
Sever response: {'game_id': 'b293fca8f0ab', 'status': 'ongoing', 'tries_remains': 4, 'word': '_ _ e t t l e _ _ _ e '}
Guessing letter: i
Sever response: {'game_id': 'b293fca8f0ab', 'status': 'ongoing', 'tries_remains': 3, 'word': '_ _ e t t l e _ _ _ e '}
Guessing letter: n
Sever response: {'game_id': 'b293fca8f0ab', 

## Playing recorded games:
Please finalize your code prior to running the cell below. Once this code executes once successfully your submission will be finalized. Our system will not allow you to rerun any additional games.

Please note that it is expected that after you successfully run this block of code that subsequent runs will result in the error message "Your account has been deactivated".

Once you've run this section of the code your submission is complete. Please send us your source code via email.

In [18]:
for i in range(1000):
    print('Playing ', i, ' th game')
    # Uncomment the following line to execute your final runs. Do not do this until you are satisfied with your submission
    api.start_game(practice=0,verbose=False)
    
    # DO NOT REMOVE as otherwise the server may lock you out for too high frequency of requests
    time.sleep(0.5)

Playing  0  th game
Playing  1  th game
Playing  2  th game
Playing  3  th game
Playing  4  th game
Playing  5  th game
Playing  6  th game
Playing  7  th game
Playing  8  th game
Playing  9  th game
Playing  10  th game
Playing  11  th game
Playing  12  th game
Playing  13  th game
Playing  14  th game
Playing  15  th game
Playing  16  th game
Playing  17  th game
Playing  18  th game
Playing  19  th game
Playing  20  th game
Playing  21  th game
Playing  22  th game
Playing  23  th game
Playing  24  th game
Playing  25  th game
Playing  26  th game
Playing  27  th game
Playing  28  th game
Playing  29  th game
Playing  30  th game
Playing  31  th game
Playing  32  th game
Playing  33  th game
Playing  34  th game
Playing  35  th game
Playing  36  th game
Playing  37  th game
Playing  38  th game
Playing  39  th game
Playing  40  th game
Playing  41  th game
Playing  42  th game
Playing  43  th game
Playing  44  th game
Playing  45  th game
Playing  46  th game
Playing  47  th game
Pl

HangmanAPIError: {'error': 'You have reached 1000 of games', 'status': 'denied'}

## To check your game statistics
1. Simply use "my_status" method.
2. Returns your total number of games, and number of wins.

In [19]:
[total_practice_runs,total_recorded_runs,total_recorded_successes,total_practice_successes] = api.my_status() # Get my game stats: (# of tries, # of wins)
success_rate = total_r
ecorded_successes/total_recorded_runs
print('overall success rate = %.3f' % success_rate)

overall success rate = 0.610
